In [1]:
import numpy as np
import pandas as pd
import requests
import requests_cache
import lxml.html as lx
import html
from bs4 import BeautifulSoup
from lxml import etree
import re
import time

import matplotlib.pyplot as plt
import seaborn as sns


requests_cache.install_cache("proj")

In [2]:
def get_used_data(zipcode):
    
    s = requests.Session()
    s.headers['User-Agent'] = """Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"""
    main_df = pd.DataFrame(columns=['name', 'link'])
    search_url = 'https://www.cars.com/shopping/results/'
    firstPage = s.get(search_url, params = {"maximum_distance":100, 
                                            "stock_type": 'used',
                                            "zip": zipcode})
    currentPage = firstPage.text # initialize 
    
    done = False
    while not done:
        
        currentHtml = lx.fromstring(currentPage)
        
        # get data ...
        vehicle_names = currentHtml.xpath(f'//a[@data-linkname="vehicle-listing"]/h2/text()')
        vehicle_links = currentHtml.xpath(f'//a[@data-linkname="vehicle-listing"]/@href')
        page_df = pd.DataFrame({'name': vehicle_names, 'link': vehicle_links})
        main_df = pd.concat([main_df, page_df], axis=0, ignore_index=True)
        
        # whether to continue
        if len(currentHtml.xpath(f'//*[@id="next_paginate"]/@href'))==0:
            done = True
        else: 
            nextLink = currentHtml.xpath(f"//*[@id='next_paginate']/@href")[0]
            nextReq = s.get(f"https://www.cars.com{nextLink}")
            currentPage = nextReq.text # update page to scrape
    return main_df

In [3]:
url_df = get_used_data('95616')

In [4]:
url_df

,name,link
0,2021 Lexus RX 350L Base,/vehicledetail/25107cf2-d55d-4abb-a6e5-5c2f2e7...
1,2022 Toyota Supra 2,/vehicledetail/59354ba1-8a99-4df4-a1e0-12057ff...
2,2020 Jeep Gladiator Sport S,/vehicledetail/41b8330e-125e-4ad8-be2a-75f140b...
3,2020 Mercedes-Benz AMG C 63 AMG C 63,/vehicledetail/7104a4f7-1bc7-427f-878c-ca66a9f...
4,2021 Mercedes-Benz GLC 300,/vehicledetail/844e6e4f-6c37-4861-9982-7638943...
...,...,...
9971,Used 2024 BMW X5 xDrive40i,/vehicledetail/3ea40a14-098e-46bb-8646-5240e0b...
9972,Used 2023 Mercedes-Benz GLC 300 4MATIC Coupe,/vehicledetail/134a96d0-b4ef-42bd-a604-fc36ed6...
9973,Certified 2020 Lincoln Navigator Reserve,/vehicledetail/273c7551-19f8-46df-b4f4-88600a1...
9974,Used 2020 Subaru Outback Touring XT,/vehicledetail/d1ca3abd-ea24-41af-b292-12a5038...


In [172]:
sample_url = '/vehicledetail/844e6e4f-6c37-4861-9982-7638943ddff4/'
# print(sample_url)

car_page = requests.get(f"https://www.cars.com{sample_url}")
car_html = lx.fromstring(car_page.text)

# feature_names = car_html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[1]/dl/dt/text()')
# feature_names

In [159]:
features = car_html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[1]/dl/dt/text()')
[x.strip() for x in features]

['Exterior color',
 'Interior color',
 'Drivetrain',
 'MPG',
 'Fuel type',
 'Transmission',
 'Engine',
 'VIN',
 'Stock #',
 'Mileage']

In [160]:
car_html.xpath('//span[@data_qa="mpg"]/text()')

[]

In [161]:
feature_names2 = car_html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[2]/dl/dt/text()')
feature_names2

['Accidents or damage', '1-owner vehicle', 'Personal use only']

In [162]:
car_html.xpath('//h1[@class="listing-title"]/text()')

['2021 Mercedes-Benz GLC 300 ']

In [164]:
features2 = car_html.xpath('//ul[@class="vehicle-features-list"]')
# for j in features2[0].iterchildren():
#     print(j.text)

In [165]:
history_names = car_html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[3]/dl/dt/text()')
history_content = car_html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[3]/dl/dd/text()')
# history_names
history_content

[]

In [166]:
car_html.xpath('//div[@class="dealer-address"]/text()')[0].split(' ')[-1]

'95825'

In [167]:
car_html.xpath('//*[@id="main-content"]/div[5]/section/header/div[2]/span[1]/text()')[0][1:]

'30,995'

In [168]:
rating_elements = car_html.xpath('//*[@id="vehicle-reviews"]/div/div[4]/ul/li')

# for elem in rating_elements:
    
#     for 

In [169]:
for i in rating_elements[0].iterchildren():
    print(i.attrib['class'])

sds-definition-list__display-name
sds-definition-list__value


In [5]:
import time

def scrape_car_info(url, local_session):
    
    full_url = f"https://www.cars.com{url}"
    info_dict = dict()
    
    time.sleep(2)
    page = local_session.get(full_url)
    html = lx.fromstring(page.text)
    
    # price in dollars
    try: info_dict['Price'] = html.xpath('//*[@id="main-content"]/div[5]/section/header/div[2]/span[1]/text()')[0][1:]
    except:
        return info_dict
        
    # basic features
    basic_feature_names = html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[1]/dl/dt/text()')
    basic_feature_content = html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[1]/dl/dd/text()')
    basic_feature_content = [x.strip() for x in basic_feature_content]
    
    for n, c in zip(basic_feature_names, basic_feature_content):
        
        if n == 'MPG':
            try:
                c = html.xpath('//span[@data_qa="mpg"]/text()')[0]
                info_dict['MPG'] = c
                
            except:
                info_dict[n] = c
                
        info_dict[n] = c
            
    # name
    info_dict['Title'] = html.xpath('//h1[@class="listing-title"]/text()')[0]
    
    # fancy features
    fancy_feature_names = html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[2]/dl/dt/text()')
    fancy_feature_elements = html.xpath('//ul[@class="vehicle-features-list"]')
    fancy_feature_content = []
    
    for elem in fancy_feature_elements:
        content_list = [x.text for x in elem.iterchildren()]
        fancy_feature_content.append('&'.join(content_list))
    
    for n, c in zip(fancy_feature_names, fancy_feature_content):
        info_dict[n] = c
        
    # car history
    history_names = html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[3]/dl/dt/text()')
    history_content = html.xpath('//*[@id="main-content"]/div[5]/div[3]/section[3]/dl/dd/text()')
    
    for n, c in zip(history_names, history_content):
        info_dict[n] = c
        
    # dealer/car address: full address
    try:
        info_dict['Address'] = html.xpath('//div[@class="dealer-address"]/text()')[0]
    except:
        info_dict['Address'] = None
    
    rating_elements = html.xpath('//*[@id="vehicle-reviews"]/div/div[4]/ul/li')

    for elem in rating_elements:
        name = 0
        val = 0
        for tag in elem.iterchildren():
            if tag.attrib['class'] == 'sds-definition-list__display-name':
                name = tag.text
            if tag.attrib['class'] == 'sds-definition-list__value':
                try:
                    val = float(tag.text)
                except:
                    val = tag.text
                    
        info_dict[f"{name} score"] = val
    
    return info_dict
    

In [173]:
s0 = requests.Session()
empty_df = pd.DataFrame()
sample_dict = scrape_car_info('/vehicledetail/844e6e4f-6c37-4861-9982-7638943ddff4/', s0)


sample_dict

# df1 = pd.DataFrame.from_dict([sample_dict])
# df2 = pd.DataFrame.from_dict([sample_dict])

# pd.concat([df1, df2], axis=0, ignore_index=True)

{'Price': '30,995',
 'Exterior color': 'Selenite Grey Metallic',
 'Interior color': 'Black MB-Tex',
 'Drivetrain': '–',
 'MPG': '–',
 'Fuel type': 'Gasoline',
 'Transmission': '–',
 'Engine': '–',
 'VIN': 'W1N0G8EB3MV324338',
 'Stock #': 'MV324338',
 'Mileage': '27,041 mi.',
 'Title': '2021 Mercedes-Benz GLC 300 ',
 'Address': '1810 Howe Avenue Sacramento, CA 95825',
 'Comfort score': 4.7,
 'Interior score': 4.7,
 'Performance score': 4.6,
 'Value score': 4.4,
 'Exterior score': 4.8,
 'Reliability score': 4.2}

In [6]:
import concurrent, threading
from tqdm import tqdm
import traceback


def get_session(thread_local):
    '''Create a new requests.Session if there is none in thread_local'''
    if not hasattr(thread_local, "session"): 
        thread_local.session = requests.Session()
    return thread_local.session

thread_local = threading.local()
local_session = get_session(thread_local)

def threading_get_features(url_df):
    
    info_df = pd.DataFrame()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        
        # map future as a function of URL to name
        future_to_url = {executor.submit(scrape_car_info, url, local_session): name for name, url in zip(url_df.name, url_df.link)}
        
        for future in tqdm(concurrent.futures.as_completed(future_to_url)):
            name = future_to_url[future]
            try:
                info_dict = future.result()
                single_info_df = pd.DataFrame.from_dict([info_dict])
                info_df = pd.concat([info_df, single_info_df], axis=0, ignore_index=True)

                
            except Exception as exc:
                print(name)
                traceback.print_exc()
                break
                
    return info_df


In [7]:
info_df = threading_get_features(url_df)

734it [02:33,  4.80it/s]


KeyboardInterrupt: 

In [181]:
info_df.to_csv("cars.csv")

In [11]:
info_df.head(30)

,Unnamed: 0,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,...,1-owner vehicle,Personal use only,Address,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall
0,0,"38,590",Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.5,4.5,4.8,4.5,4.8,4.8,NaN
1,1,"44,490",Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",2.7,2.8,3.3,2.5,3.2,2.7,NaN
2,2,"54,495",Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,...,NaN,NaN,"1810 Howe Avenue Sacramento, CA 95825",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"32,990",Firecracker Red Clearcoat,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"3.6L V-6 DOHC, variable valve control, regular...",1C6HJTAG3LL121323,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.7,4.7,4.6,4.2,4.8,4.6,NaN
4,4,"30,995",Selenite Grey Metallic,Black MB-Tex,–,–,Gasoline,–,–,W1N0G8EB3MV324338,...,NaN,NaN,"1810 Howe Avenue Sacramento, CA 95825",4.7,4.7,4.6,4.4,4.8,4.2,NaN
5,5,"59,990",Polar White,–,Rear-wheel Drive,17–24,Gasoline,Automatic,"AMG 4L V-8 gasoline direct injection, DOHC, va...",WDDWK8GB6LF969321,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.5,5.0,5.0,4.0,5.0,4.5,NaN
6,6,"35,655",Summit White,Jet Black,Four-wheel Drive,17–24,Gasoline,Automatic,"3.6L V-6 gasoline direct injection, DOHC, vari...",1GCGTDEN0N1295183,...,Yes,No,"2449 Fulton Avenue Sacramento, CA 95825",4.5,4.3,4.1,4.2,4.8,4.3,NaN
7,7,"45,301",Crystal White Tricoat,Jet Black w/Jet Black Accents,Rear-wheel Drive,23–30,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",1G6DU5RK4P0144266,...,Yes,Yes,"2449 Fulton Avenue Sacramento, CA 95825",4.7,3.7,5.0,4.7,5.0,4.0,NaN
8,8,"50,824",Moonbow Blue,Java,All-wheel Drive,20–25,Gasoline,9-Speed Automatic,3.5L V6 24V GDI DOHC,5N1DL1HU9NC333773,...,Yes,Yes,"2020 Fulton Ave Sacramento, CA 95825",5.0,5.0,5.0,4.7,5.0,5.0,At least 1 open recall reported
9,9,"41,674",Opal White Pearl,Titan Black,All-wheel Drive,22–31,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WVWTR7ANXPE003345,...,Yes,Yes,"2701 Arden Way Sacramento, CA 95825",NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Additional Scrapping

In [12]:
info_df = pd.read_csv("cars.csv")
info_df.head(30)

,Unnamed: 0,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,...,1-owner vehicle,Personal use only,Address,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall
0,0,"38,590",Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.5,4.5,4.8,4.5,4.8,4.8,NaN
1,1,"44,490",Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",2.7,2.8,3.3,2.5,3.2,2.7,NaN
2,2,"54,495",Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,...,NaN,NaN,"1810 Howe Avenue Sacramento, CA 95825",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"32,990",Firecracker Red Clearcoat,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"3.6L V-6 DOHC, variable valve control, regular...",1C6HJTAG3LL121323,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.7,4.7,4.6,4.2,4.8,4.6,NaN
4,4,"30,995",Selenite Grey Metallic,Black MB-Tex,–,–,Gasoline,–,–,W1N0G8EB3MV324338,...,NaN,NaN,"1810 Howe Avenue Sacramento, CA 95825",4.7,4.7,4.6,4.4,4.8,4.2,NaN
5,5,"59,990",Polar White,–,Rear-wheel Drive,17–24,Gasoline,Automatic,"AMG 4L V-8 gasoline direct injection, DOHC, va...",WDDWK8GB6LF969321,...,No,Yes,"671 Orange Dr Vacaville, CA 95687",4.5,5.0,5.0,4.0,5.0,4.5,NaN
6,6,"35,655",Summit White,Jet Black,Four-wheel Drive,17–24,Gasoline,Automatic,"3.6L V-6 gasoline direct injection, DOHC, vari...",1GCGTDEN0N1295183,...,Yes,No,"2449 Fulton Avenue Sacramento, CA 95825",4.5,4.3,4.1,4.2,4.8,4.3,NaN
7,7,"45,301",Crystal White Tricoat,Jet Black w/Jet Black Accents,Rear-wheel Drive,23–30,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",1G6DU5RK4P0144266,...,Yes,Yes,"2449 Fulton Avenue Sacramento, CA 95825",4.7,3.7,5.0,4.7,5.0,4.0,NaN
8,8,"50,824",Moonbow Blue,Java,All-wheel Drive,20–25,Gasoline,9-Speed Automatic,3.5L V6 24V GDI DOHC,5N1DL1HU9NC333773,...,Yes,Yes,"2020 Fulton Ave Sacramento, CA 95825",5.0,5.0,5.0,4.7,5.0,5.0,At least 1 open recall reported
9,9,"41,674",Opal White Pearl,Titan Black,All-wheel Drive,22–31,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WVWTR7ANXPE003345,...,Yes,Yes,"2701 Arden Way Sacramento, CA 95825",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = info_df.copy(deep=True)
df = df.dropna(subset=['Price', 'VIN'])
df = df.drop(df.index[1854]) # manual handling of some car

df.isna().sum()

Unnamed: 0                0
Price                     0
Exterior color            0
Interior color            0
Drivetrain                0
MPG                     628
Fuel type               562
Transmission              0
Engine                    0
VIN                       0
Stock #                  75
Mileage                   0
Title                     0
Convenience             440
Entertainment           570
Exterior                877
Safety                  307
Seating                3645
Accidents or damage     441
Clean title            7246
1-owner vehicle         473
Personal use only       441
Address                  94
Comfort score           563
Interior score          563
Performance score       563
Value score             563
Exterior score          563
Reliability score       563
Open recall            8533
dtype: int64

## Get a full list of car brands

In [14]:
cars_url = 'https://totallossappraisals.com/the-list-of-car-brands-available-in-america-in-2023/'

cars_response = requests.get(cars_url)
cars_response.raise_for_status()
cars_html = lx.fromstring(cars_response.text)
car_brands = cars_html.xpath('/html/body/div[2]/div[1]/div/div[5]/div/ul/li/span/text()')
car_brands2 = [x.lower() for x in car_brands]

In [15]:
import traceback
# count = 0


brands = []

for i in range(len(df.Title)):
    try:
        t = df.Title.iloc[i]
        phrases = t.split(' ')
        brand_kw = phrases[1]

        for b in car_brands2:
            if brand_kw.lower() in b:
                brands.append(b)
#                 if i != count:
#                     print(i)
#                     break
#                 count += 1
            elif brand_kw == 'Datsun':
                brands.append('datsun')
                
    except Exception as e:
        df = df.drop(df.index[i])
#         traceback.print_exc()

        

df['Brand'] = brands
df

,Unnamed: 0,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,...,Personal use only,Address,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall,Brand
0,0,"38,590",Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,...,Yes,"671 Orange Dr Vacaville, CA 95687",4.5,4.5,4.8,4.5,4.8,4.8,NaN,lexus
1,1,"44,490",Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,...,Yes,"671 Orange Dr Vacaville, CA 95687",2.7,2.8,3.3,2.5,3.2,2.7,NaN,toyota
2,2,"54,495",Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,...,NaN,"1810 Howe Avenue Sacramento, CA 95825",NaN,NaN,NaN,NaN,NaN,NaN,NaN,mercedes-benz
3,3,"32,990",Firecracker Red Clearcoat,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"3.6L V-6 DOHC, variable valve control, regular...",1C6HJTAG3LL121323,...,Yes,"671 Orange Dr Vacaville, CA 95687",4.7,4.7,4.6,4.2,4.8,4.6,NaN,jeep
4,4,"30,995",Selenite Grey Metallic,Black MB-Tex,–,–,Gasoline,–,–,W1N0G8EB3MV324338,...,NaN,"1810 Howe Avenue Sacramento, CA 95825",4.7,4.7,4.6,4.4,4.8,4.2,NaN,mercedes-benz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9971,"65,991",Alpine White,Silverstone,All-wheel Drive,23–27,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UX23EU09R9T29410,...,No,"1 Geary Plaza Seaside, CA 93955",4.8,5.0,5.0,4.0,5.0,4.2,At least 1 open recall reported,bmw
9972,9972,"49,998",Black,–,All-wheel Drive,20–26,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,W1N0J8EB6PG126055,...,Yes,"195 E Auto Center Dr Fresno, CA 93710",5.0,5.0,5.0,4.5,5.0,5.0,NaN,mercedes-benz
9973,9973,"48,299",Blue Diamond,Cappuccino,Four-wheel Drive,16–21,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,5LMJJ2LT8LEL16331,...,Yes,"195 E Auto Center Dr Fresno, CA 93710",4.9,4.8,4.5,4.3,4.7,4.2,NaN,lincoln
9974,9974,"29,080",Autumn Green Metallic,Java,All-wheel Drive,23–30,Gasoline,Automatic CVT,2.4L H4 16V GDI DOHC Turbo,4S4BTGPD4L3225943,...,Yes,"5580 North Blackstone Ave Fresno, CA 93710",4.8,4.7,4.6,4.6,4.8,4.6,NaN,subaru


In [16]:
df.columns

Index(['Unnamed: 0', 'Price', 'Exterior color', 'Interior color', 'Drivetrain',
       'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN', 'Stock #',
       'Mileage', 'Title', 'Convenience', 'Entertainment', 'Exterior',
       'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Address', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall', 'Brand'],
      dtype='object')

## Scrapping from Car API

In [17]:
key = '3872f645d8mshe181db1235aa1dap1cdf69jsn255286a90aa3'
api_url = 'https://vindecoder.p.rapidapi.com/decode_vin'


VIN = '4F2YU09161KM33122'
params = {'vin': VIN}

headers = {
    'X-RapidAPI-Key': key,
    'X-RapidAPI-Host': 'vindecoder.p.rapidapi.com'
  }
    
response = requests.get(api_url, params = params, headers = headers)

In [18]:
df.VIN[0]

'JTJHZKEAXM2019071'

In [19]:
import requests,json;

VIN = df.VIN[0]
url = f'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{VIN}?format=xml'
r = requests.get(url);


soup = BeautifulSoup(r.text)
print(soup.prettify())
api_html = lx.fromstring(r.text)

<html>
 <body>
  <response xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
   <count>
    136
   </count>
   <message>
    Results returned successfully. NOTE: Any missing decoded values should be interpreted as NHTSA does not have data on the specific variable. Missing value should NOT be interpreted as an indication that a feature or technology is unavailable for a vehicle.
   </message>
   <searchcriteria>
    VIN:JTJHZKEAXM2019071
   </searchcriteria>
   <results>
    <decodedvariable>
     <variableid>
      142
     </variableid>
     <variable>
      Suggested VIN
     </variable>
     <valueid>
     </valueid>
     <value>
     </value>
    </decodedvariable>
    <decodedvariable>
     <variableid>
      143
     </variableid>
     <variable>
      Error Code
     </variable>
     <valueid>
      0
     </valueid>
     <value>
      0
     </value>
    </decodedvariable>
    <decodedvariable>
     <variableid>
      144
     <

In [54]:
api_html = lx.fromstring(r.text)


def get_api_attr(html, var_name):
    try:
        attr_value = html.xpath(f'//decodedvariable[variable/text()="{var_name}"]/value/text()')[0]
        return attr_value
    except:
        return np.nan
    
def get_attr_by_vin(VIN):
    
    url = f'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{VIN}?format=xml'
    try:
        response = requests.get(url)
    except:
        print("Failed to get data.")
        return
    
    api_html = lx.fromstring(response.text)
    err_code = api_html.xpath('//decodedvariable[variable/text()="Error Code"]/value/text()')[0]
    
    attr_list = ['Vehicle Type', 'Trim', 'Make', 'Model', 'Model Year', 'Series', "Displacement (L)", "Engine Brake (hp) From",
            'Engine Number of Cylinders', 'Plant Country', 'Body Class']
        
    attr_dict = dict()
    
    if err_code == "0":
        for attr in attr_list:
            attr_val = get_api_attr(api_html, attr)
            attr_dict[attr] = attr_val
            
        return attr_dict
    
    else:
        print(VIN)
        print(err_code)
        for attr in attr_list:
            attr_dict[attr] = np.nan

        return attr_dict


# api_html.xpath('//decodedvariable[variable/text()="Error Code"]/value/text()')[0]

testvin = df.VIN[0]
s0 = requests.Session()


In [55]:
testvin

'JTJHZKEAXM2019071'

In [56]:
pd.DataFrame([get_attr_by_vin(testvin)])

,Vehicle Type,Trim,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),Premium,LEXUS,RX,2021,GGL21L,3.5,290,6,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...


In [57]:
attr_list = pd.DataFrame([get_attr_by_vin(testvin)]).columns

In [58]:
VINs = df.VIN

api_df = pd.DataFrame(columns = attr_list)

for VIN in tqdm(VINs):
    
    attr_dict = get_attr_by_vin(VIN)
    api_df = pd.concat([api_df, pd.DataFrame([attr_dict])], axis=0)
    
    time.sleep(0.5)

    
    

  1%|▏         | 134/9903 [01:11<1:26:43,  1.88it/s]

9110201028
6,7


  6%|▌         | 596/9903 [05:21<1:23:57,  1.85it/s]

ZFFLA40B000092402
1,11,14,400


  7%|▋         | 654/9903 [05:52<1:22:19,  1.87it/s]

...

 88%|████████▊ | 8677/9903 [1:19:18<10:55,  1.87it/s]

1Z8749S407741
5,6,14


 95%|█████████▍| 9371/9903 [1:25:37<04:47,  1.85it/s]

WPOZZZ91ZDS100622
1,7,400


100%|██████████| 9903/9903 [1:30:28<00:00,  1.82it/s]


In [59]:
api_df.to_csv('api_df.csv')
api_df

,Vehicle Type,Trim,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),Premium,LEXUS,RX,2021,GGL21L,3.5,290,6,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...
0,PASSENGER CAR,NaN,TOYOTA,Supra,2022,Supra 2.0,2.0,255,4,AUSTRIA,Coupe
0,PASSENGER CAR,NaN,MERCEDES-BENZ,E-Class,2023,E350 4MATIC,2,255,4,GERMANY,Sedan/Saloon
0,TRUCK,NaN,JEEP,Gladiator,2020,Sport,3.6,NaN,6,UNITED STATES (USA),Pickup
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),GLC300-4M,MERCEDES-BENZ,GLC-Class,2021,NaN,2.0,NaN,4,FINLAND,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...
...,...,...,...,...,...,...,...,...,...,...,...
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),X5 xDrive40i,BMW,X5,2024,NaN,3,375,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,MERCEDES-BENZ,GLC-Class,2023,GLC300 4MATIC,2.0,255,4,GERMANY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,LINCOLN,Navigator,2020,Reserve,3.5,310,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),Touring+M/R+NAVI(HK),SUBARU,Outback,2020,NaN,2.4,260,4,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...


In [237]:
api_df = api_df.reset_index(drop=True)
df = df.reset_index(drop=True)

df_combined = pd.concat([api_df, df], axis=1)
df_combined

,Vehicle Type,Trim,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,distance,Speed,Lap
0,MULTIPURPOSE PASSENGER VEHICLE (MPV),Premium,LEXUS,RX,2021,GGL21L,3.5,290,6,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,1.0,1.0,Lap 1
1,PASSENGER CAR,NaN,TOYOTA,Supra,2022,Supra 2.0,2.0,255,4,AUSTRIA,Coupe,2.0,2.0,Lap 1
2,PASSENGER CAR,NaN,MERCEDES-BENZ,E-Class,2023,E350 4MATIC,2,255,4,GERMANY,Sedan/Saloon,3.0,3.0,Lap 1
3,TRUCK,NaN,JEEP,Gladiator,2020,Sport,3.6,NaN,6,UNITED STATES (USA),Pickup,1.0,1.2,Lap 2
4,MULTIPURPOSE PASSENGER VEHICLE (MPV),GLC300-4M,MERCEDES-BENZ,GLC-Class,2021,NaN,2.0,NaN,4,FINLAND,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,2.0,2.2,Lap 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,MULTIPURPOSE PASSENGER VEHICLE (MPV),X5 xDrive40i,BMW,X5,2024,NaN,3,375,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,NaN
9899,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,MERCEDES-BENZ,GLC-Class,2023,GLC300 4MATIC,2.0,255,4,GERMANY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,NaN
9900,MULTIPURPOSE PASSENGER VEHICLE (MPV),NaN,LINCOLN,Navigator,2020,Reserve,3.5,310,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,NaN
9901,MULTIPURPOSE PASSENGER VEHICLE (MPV),Touring+M/R+NAVI(HK),SUBARU,Outback,2020,NaN,2.4,260,4,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,NaN,NaN,NaN


In [68]:
df01 = df_combined.copy(deep=True)

df01 = df01.drop(columns = ['Brand', 'Address', 'Vehicle Type', 'Trim'])
df01.fillna(pd.NA, inplace=True)

In [69]:
df01.columns

Index(['Make', 'Model', 'Model Year', 'Series', 'Displacement (L)',
       'Engine Brake (hp) From', 'Engine Number of Cylinders', 'Plant Country',
       'Body Class', 'Unnamed: 0', 'Price', 'Exterior color', 'Interior color',
       'Drivetrain', 'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN',
       'Stock #', 'Mileage', 'Title', 'Convenience', 'Entertainment',
       'Exterior', 'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall'],
      dtype='object')

In [70]:
df01["Personal use only"].value_counts()

Yes    6202
No     3260
Name: Personal use only, dtype: int64

In [71]:
brand_country = requests.get('https://www.canstarblue.com.au/vehicles/car-country-of-origin/').text
brand_country_df = pd.read_html(brand_country)[0]
brand_country_df['Car Brand'] = [x.lower() for x in brand_country_df['Car Brand']]

country_dict = dict()

for b, c in zip(brand_country_df['Car Brand'], brand_country_df['Country of Origin']):
    country_dict[b] = c
    
country_dict['acura'] = 'Japan'
country_dict['gmc'] = 'America'
country_dict['ram'] = 'America'
country_dict['lincoln'] = 'America'
country_dict['buick'] = 'America'
country_dict['rivian'] = 'America'
country_dict['fisker'] = 'America'
country_dict['lucid'] = 'America'
country_dict['hummer'] = 'America'
country_dict['polestar'] = 'Sweden'

ori_countries = []

for brand in df01['Make']:
    try:
        brand = brand.lower()
        country = country_dict[brand]
        ori_countries.append(country)
    except:
        ori_countries.append(pd.NA)
        
df01['Country of Origin'] = ori_countries

In [72]:
pd.set_option('display.max_columns', None)
df01

,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,Unnamed: 0,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Mileage,Title,Convenience,Entertainment,Exterior,Safety,Seating,Accidents or damage,Clean title,1-owner vehicle,Personal use only,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall,Country of Origin
0,LEXUS,RX,2021,GGL21L,3.5,290,6,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,0,"38,590",Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,PM2019071,"35,626 mi.",2021 Lexus RX 350L Base,Adaptive Cruise Control&Keyless Start&Navigati...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Sunroof/Moonroof,Automatic Emergency Braking&Backup Camera&Blin...,Third Row Seating,None reported,Yes,No,Yes,4.5,4.5,4.8,4.5,4.8,4.8,<NA>,Japan
1,TOYOTA,Supra,2022,Supra 2.0,2.0,255,4,AUSTRIA,Coupe,1,"44,490",Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,PNW052317,"10,810 mi.",2022 Toyota Supra 2,Adaptive Cruise Control&Keyless Start&Navigati...,Bluetooth®&HomeLink&USB Port,Alloy Wheels,Automatic Emergency Braking&Backup Camera&Blin...,Leather Seats,None reported,Yes,No,Yes,2.7,2.8,3.3,2.5,3.2,2.7,<NA>,Japan
2,MERCEDES-BENZ,E-Class,2023,E350 4MATIC,2,255,4,GERMANY,Sedan/Saloon,2,"54,495",Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,PB105789P,"10,859 mi.",2023 Mercedes-Benz E-Class,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Germany
3,JEEP,Gladiator,2020,Sport,3.6,<NA>,6,UNITED STATES (USA),Pickup,3,"32,990",Firecracker Red Clearcoat,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"3.6L V-6 DOHC, variable valve control, regular...",1C6HJTAG3LL121323,TLL121323,"46,608 mi.",2020 Jeep Gladiator Sport S,Adaptive Cruise Control&Keyless Start&Navigati...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Tow Hitch&Tow Hooks,Automatic Emergency Braking&Backup Camera&Blin...,<NA>,None reported,Yes,No,Yes,4.7,4.7,4.6,4.2,4.8,4.6,<NA>,America
4,MERCEDES-BENZ,GLC-Class,2021,<NA>,2.0,<NA>,4,FINLAND,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,4,"30,995",Selenite Grey Metallic,Black MB-Tex,–,–,Gasoline,–,–,W1N0G8EB3MV324338,MV324338,"27,041 mi.",2021 Mercedes-Benz GLC 300,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4.7,4.7,4.6,4.4,4.8,4.2,<NA>,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,BMW,X5,2024,<NA>,3,375,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,9971,"65,991",Alpine White,Silverstone,All-wheel Drive,23–27,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UX23EU09R9T29410,R9T29410BR,"4,185 mi.",2024 BMW X5 xDrive40i,Adaptive Cruise Control&Heated Seats&Heated St...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Sunroof/Moonroof,Backup Camera&Blind Spot Monitor&Brake Assist&...,Leather Seats&Memory Seat,None reported,<NA>,No,No,4.8,5.0,5.0,4.0,5.0,4.2,At least 1 open recall reported,Germany
9899,MERCEDES-BENZ,GLC-Class,2023,GLC300 4MATIC,2.0,255,4,GERMANY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,9972,"49,998",Black,–,All-wheel Drive,20–26,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,W1N0J8EB6PG126055,PG126055PL,"11,660 mi.",2023 Mercedes-Benz GLC 300 4MATIC Coupe,Adaptive Cruise Control&Heated Seats&Keyless S...,Bluetooth®&HomeLink&Premium Sound System,Alloy Wheels&Sunroof/Moonroof,Backup Camera&Blind Spot Monitor&Brake Assist&...,Leather Seats&Memory Seat,None reported,<NA>,Yes,Yes,5.0,5.0,5.0,4.5,5.0,5.0,<NA>,Germany
9900,LINCOLN,Navigator,2020,Reserve,3.5,310,6,UNITED STATES (USA),Sport Utility Vehicle (SU

In [73]:
mpg_city = []
mpg_hw = []

for mpg in df01.MPG:
    try:
        mpgs = mpg.split('–')
        if not (mpgs[0] in ['Unknown', '', '0'] or mpgs[1] == '0'):
            mpg_city.append(mpgs[0])
            mpg_hw.append(mpgs[1])
        else:
            mpg_city.append(pd.NA)
            mpg_hw.append(pd.NA)
    except:
        mpg_city.append(pd.NA)
        mpg_hw.append(pd.NA)
        
df01['MPG City'] = mpg_city
df01['MPG Highway'] = mpg_hw

In [74]:
df01['Price'] = df01['Price'].apply(lambda x: int(x.replace(',', '')))

In [75]:
df01.columns

Index(['Make', 'Model', 'Model Year', 'Series', 'Displacement (L)',
       'Engine Brake (hp) From', 'Engine Number of Cylinders', 'Plant Country',
       'Body Class', 'Unnamed: 0', 'Price', 'Exterior color', 'Interior color',
       'Drivetrain', 'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN',
       'Stock #', 'Mileage', 'Title', 'Convenience', 'Entertainment',
       'Exterior', 'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall', 'Country of Origin', 'MPG City',
       'MPG Highway'],
      dtype='object')

In [76]:
df01['Body Class'].replace({'Incomplete - Chassis Cab (Double Cab)': pd.NA, 
                           'Incomplete':pd.NA, 'Incomplete - Chassis Cab (Single Cab)': pd.NA,
                           'Incomplete - Chassis Cab (Number of Cab Unknown)':pd.NA}, inplace=True)
df01['Body Class'].value_counts()

Sport Utility Vehicle (SUV)/Multi-Purpose Vehicle (MPV)    4335
Sedan/Saloon                                               2495
Pickup                                                     1124
Hatchback/Liftback/Notchback                                621
Coupe                                                       466
Convertible/Cabriolet                                       253
Minivan                                                     210
Cargo Van                                                   115
Wagon                                                        81
Crossover Utility Vehicle (CUV)                              76
Van                                                          47
Roadster                                                     22
Sport Utility Truck (SUT)                                     4
Truck                                                         1
Name: Body Class, dtype: int64

In [77]:
df01.isna().sum()

Make                            35
Model                           35
Model Year                      35
Series                        2972
Displacement (L)               618
Engine Brake (hp) From        3064
Engine Number of Cylinders    1313
Plant Country                   35
Body Class                      53
Unnamed: 0                       0
Price                            0
Exterior color                   0
Interior color                   0
Drivetrain                       0
MPG                            628
Fuel type                      562
Transmission                     0
Engine                           0
VIN                              0
Stock #                         75
Mileage                          0
Title                            0
Convenience                    440
Entertainment                  570
Exterior                       877
Safety                         307
Seating                       3645
Accidents or damage            441
Clean title         

In [78]:
df01

,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,Unnamed: 0,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Mileage,Title,Convenience,Entertainment,Exterior,Safety,Seating,Accidents or damage,Clean title,1-owner vehicle,Personal use only,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall,Country of Origin,MPG City,MPG Highway
0,LEXUS,RX,2021,GGL21L,3.5,290,6,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,0,38590,Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,PM2019071,"35,626 mi.",2021 Lexus RX 350L Base,Adaptive Cruise Control&Keyless Start&Navigati...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Sunroof/Moonroof,Automatic Emergency Braking&Backup Camera&Blin...,Third Row Seating,None reported,Yes,No,Yes,4.5,4.5,4.8,4.5,4.8,4.8,<NA>,Japan,19,26
1,TOYOTA,Supra,2022,Supra 2.0,2.0,255,4,AUSTRIA,Coupe,1,44490,Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,PNW052317,"10,810 mi.",2022 Toyota Supra 2,Adaptive Cruise Control&Keyless Start&Navigati...,Bluetooth®&HomeLink&USB Port,Alloy Wheels,Automatic Emergency Braking&Backup Camera&Blin...,Leather Seats,None reported,Yes,No,Yes,2.7,2.8,3.3,2.5,3.2,2.7,<NA>,Japan,25,32
2,MERCEDES-BENZ,E-Class,2023,E350 4MATIC,2,255,4,GERMANY,Sedan/Saloon,2,54495,Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,PB105789P,"10,859 mi.",2023 Mercedes-Benz E-Class,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Germany,<NA>,<NA>
3,JEEP,Gladiator,2020,Sport,3.6,<NA>,6,UNITED STATES (USA),Pickup,3,32990,Firecracker Red Clearcoat,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"3.6L V-6 DOHC, variable valve control, regular...",1C6HJTAG3LL121323,TLL121323,"46,608 mi.",2020 Jeep Gladiator Sport S,Adaptive Cruise Control&Keyless Start&Navigati...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Tow Hitch&Tow Hooks,Automatic Emergency Braking&Backup Camera&Blin...,<NA>,None reported,Yes,No,Yes,4.7,4.7,4.6,4.2,4.8,4.6,<NA>,America,17,22
4,MERCEDES-BENZ,GLC-Class,2021,<NA>,2.0,<NA>,4,FINLAND,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,4,30995,Selenite Grey Metallic,Black MB-Tex,–,–,Gasoline,–,–,W1N0G8EB3MV324338,MV324338,"27,041 mi.",2021 Mercedes-Benz GLC 300,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4.7,4.7,4.6,4.4,4.8,4.2,<NA>,Germany,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,BMW,X5,2024,<NA>,3,375,6,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,9971,65991,Alpine White,Silverstone,All-wheel Drive,23–27,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UX23EU09R9T29410,R9T29410BR,"4,185 mi.",2024 BMW X5 xDrive40i,Adaptive Cruise Control&Heated Seats&Heated St...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Sunroof/Moonroof,Backup Camera&Blind Spot Monitor&Brake Assist&...,Leather Seats&Memory Seat,None reported,<NA>,No,No,4.8,5.0,5.0,4.0,5.0,4.2,At least 1 open recall reported,Germany,23,27
9899,MERCEDES-BENZ,GLC-Class,2023,GLC300 4MATIC,2.0,255,4,GERMANY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,9972,49998,Black,–,All-wheel Drive,20–26,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,W1N0J8EB6PG126055,PG126055PL,"11,660 mi.",2023 Mercedes-Benz GLC 300 4MATIC Coupe,Adaptive Cruise Control&Heated Seats&Keyless S...,Bluetooth®&HomeLink&Premium Sound System,Alloy Wheels&Sunroof/Moonroof,Backup Camera&Blind Spot Monitor&Brake Assist&...,Leather Seats&Memory Seat,None reported,<NA>,Yes,Yes,5.0,5.0,5.0,4.5,5.0,5.0,<NA>,Germany,20,26
9900,LINCOLN,Navigator,2020,Reser

In [79]:
def get_mileage(s):
    new_str = re.sub(',|( mi\.)', '', s)
    return int(new_str)

mileages = []

for m in df01.Mileage:
    try:
        mileages.append(get_mileage(m))
    except:
        mileages.append(pd.NA)
df01.Mileage = mileages

In [80]:
df01.to_csv('cleaned_data.csv')

# Visualization

- Depreciation Factor: Year and Millage
    * Price vs. Year
     * Car Type, Country of Origin, etc.
    * Price vs. Millage
     * Car Type, Country of Origin, etc.
     
- Price vs. Performance
 * By Engine Power (hp), Drivetrain, Displacement (L)
 
- Price vs. Car History
    * Whether 1-owner? Accident?

- Rating Scores
    * This motivates more explorations... --> NLP 

    
- Other factors
    * additional features
        * convenience
        * entertainment
        * safety
        

- Sensitivity Check based on missingness: are data missing by random?
    * ANOVA based on missingness in each column




    

In [329]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'

df01 = pd.read_csv('cleaned_data.csv')
df01.drop(columns=['Unnamed: 0','Unnamed: 0.1'], inplace=True)
# df01['Model Year'] = df01['Model Year'].astype(int)

def get_model_prob(make, model, year):
    
    make_df = df01[df01['Make'] == make]
    model_df = make_df[(make_df['Model']==model) & (make_df['Model Year']==year)]
    
    prop = len(model_df) / len(make_df)
    
    return prop

model_wt_df = pd.DataFrame(columns=['Make','Model','Year','Weight'])

for make in list(df01.Make.unique()):
    make_models = df01[df01['Make']==make]['Model'].unique()
    for model in make_models:
        years = df01[(df01['Make']==make)&(df01['Model']==model)]['Model Year'].unique()
        for year in years:
            wt = get_model_prob(make, model, year)
            model_wt_df = pd.concat([model_wt_df, pd.DataFrame([{'Make': make, 'Model': model,'Year':year, 'Weight':wt}])])

            

#         wt = get_model_prob(make, model)
#         model_wt_df = pd.concat([model_wt_df, pd.DataFrame([{'Make': make, 'Model': model, 'Weight':wt}])])
        

In [330]:
model_wt_df['Inverse Weight'] = model_wt_df['Weight']**(-1)
model_wt_df.head(4)

,Make,Model,Year,Weight,Inverse Weight
0,LEXUS,RX,2021.0,0.038732,25.818182
0,LEXUS,RX,2018.0,0.017606,56.800000
0,LEXUS,RX,2019.0,0.010563,94.666667
0,LEXUS,RX,2013.0,0.028169,35.500000


In [338]:
iws = []
make_iw_sum = []

for idx in df01.index:
    sub_df = df01.loc[idx]
    make, model, year = sub_df['Make'], sub_df['Model'], sub_df['Model Year']
    iw = model_wt_df[(model_wt_df['Make'] == make) & (model_wt_df['Model'] == model)
                     & (model_wt_df['Year'] == year)]['Inverse Weight']
    iw_sum = model_wt_df[model_wt_df['Make'] == make]['Inverse Weight'].sum()
    try:
        iw = iw[0]/iw_sum
    except:
        iw = pd.NA
    iws.append(iw)

df01['Weights'] = iws
df01['WxP'] = df01['Weights'] * df01['Price']
df01.head(3)


,Unnamed: 0,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,Price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Mileage,Title,Convenience,Entertainment,Exterior,Safety,Seating,Accidents or damage,Clean title,1-owner vehicle,Personal use only,Comfort score,Interior score,Performance score,Value score,Exterior score,Reliability score,Open recall,Country of Origin,MPG City,MPG Highway,Weights,WxP
0,0,LEXUS,RX,2021.0,GGL21L,3.5,290.0,6.0,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,38590,Eminent White Pearl,–,Front-wheel Drive,19–26,Gasoline,Automatic,"3.5L V-6 port/direct injection, DOHC, VVT-iW v...",JTJHZKEAXM2019071,PM2019071,35626,2021 Lexus RX 350L Base,Adaptive Cruise Control&Keyless Start&Navigati...,Apple CarPlay®/Android Auto®&Bluetooth®&HomeLi...,Alloy Wheels&Sunroof/Moonroof,Automatic Emergency Braking&Backup Camera&Blin...,Third Row Seating,None reported,Yes,No,Yes,4.5,4.5,4.8,4.5,4.8,4.8,NaN,Japan,19.0,26.0,0.001201,46.329757
1,1,TOYOTA,Supra,2022.0,Supra 2.0,2.0,255.0,4.0,AUSTRIA,Coupe,44490,Nitro Yellow,Black,Rear-wheel Drive,25–32,Gasoline,Automatic,"2L I-4 gasoline direct injection, DOHC, variab...",WZ1DB2C00NW052317,PNW052317,10810,2022 Toyota Supra 2,Adaptive Cruise Control&Keyless Start&Navigati...,Bluetooth®&HomeLink&USB Port,Alloy Wheels,Automatic Emergency Braking&Backup Camera&Blin...,Leather Seats,None reported,Yes,No,Yes,2.7,2.8,3.3,2.5,3.2,2.7,NaN,Japan,25.0,32.0,0.003795,168.84029
2,2,MERCEDES-BENZ,E-Class,2023.0,E350 4MATIC,2.0,255.0,4.0,GERMANY,Sedan/Saloon,54495,Cirrus Silver Metallic,Neva White/Magma Grey MB-Tex,–,–,Gasoline,–,–,W1KZF8EB8PB105789,PB105789P,10859,2023 Mercedes-Benz E-Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germany,NaN,NaN,0.002883,157.123916


In [352]:
df01.to_csv('cleaned_df2.csv', index=False)

# Import data for visualization

In [2]:
df01 = pd.read_csv('cleaned_df2.csv')

In [3]:
df01

,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,Price,...,Performance score,Value score,Exterior score,Reliability score,Open recall,Country of Origin,MPG City,MPG Highway,Weights,WxP
0,LEXUS,RX,2021.0,GGL21L,3.5,290.0,6.0,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,38590,...,4.8,4.5,4.8,4.8,NaN,Japan,19.0,26.0,0.001201,46.329757
1,TOYOTA,Supra,2022.0,Supra 2.0,2.0,255.0,4.0,AUSTRIA,Coupe,44490,...,3.3,2.5,3.2,2.7,NaN,Japan,25.0,32.0,0.003795,168.840290
2,MERCEDES-BENZ,E-Class,2023.0,E350 4MATIC,2.0,255.0,4.0,GERMANY,Sedan/Saloon,54495,...,NaN,NaN,NaN,NaN,NaN,Germany,NaN,NaN,0.002883,157.123916
3,JEEP,Gladiator,2020.0,Sport,3.6,NaN,6.0,UNITED STATES (USA),Pickup,32990,...,4.6,4.2,4.8,4.6,NaN,America,17.0,22.0,0.014186,468.011325
4,MERCEDES-BENZ,GLC-Class,2021.0,NaN,2.0,NaN,4.0,FINLAND,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,30995,...,4.6,4.4,4.8,4.2,NaN,Germany,NaN,NaN,0.001281,39.718676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,BMW,X5,2024.0,NaN,3.0,375.0,6.0,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,65991,...,5.0,4.0,5.0,4.2,At least 1 open recall reported,Germany,23.0,27.0,0.000699,46.153072
9899,MERCEDES-BENZ,GLC-Class,2023.0,GLC300 4MATIC,2.0,255.0,4.0,GERMANY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,49998,...,5.0,4.5,5.0,5.0,NaN,Germany,20.0,26.0,0.001442,72.078921
9900,LINCOLN,Navigator,2020.0,Reserve,3.5,310.0,6.0,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,48299,...,4.5,4.3,4.7,4.2,NaN,America,16.0,21.0,0.008368,404.175732
9901,SUBARU,Outback,2020.0,NaN,2.4,260.0,4.0,UNITED STATES (USA),Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,29080,...,4.6,4.6,4.8,4.6,NaN,Japan,23.0,30.0,0.002531,73.607615


### Overall Price Distribution

In [209]:
import plotly.express as px
import plotly.figure_factory as ff

temp_df = df01.dropna(subset=['Country of Origin'])
temp_df = temp_df[temp_df['Model Year'] >2015]
temp_df['sqrtPrice'] = np.log(temp_df['Price'])

# temp_df = temp_df[temp_df.Price<200000]


countries = temp_df['Country of Origin'].unique()

hist_data_by_country = [list(temp_df.sqrtPrice.values[temp_df['Country of Origin'] == c]) for c in countries]


fig = ff.create_distplot(hist_data_by_country, countries, show_hist=False)
fig.update_layout(
    title_text="Price Distribution vs. Country of Origin",
    xaxis_title="Log-Price",
    yaxis_title="Kernel Density Estimation",
    legend_title="Country of Origin"
)
fig.show()

In [5]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'

In [210]:

temp_df = df01.dropna(subset=['Plant Country'])
temp_df['Plant Country'] = temp_df['Plant Country'].replace({'ENGLAND':"UNITED KINGDOM (UK)"})
temp_df['sqrtPrice'] = np.sqrt(temp_df['Price'])


countries = df01.dropna(subset=['Plant Country'])['Plant Country'].value_counts().reset_index()['index']

fig = go.Figure()

for ct in countries:       
    fig.add_trace(go.Box(y=np.log(temp_df['Price'][(temp_df['Plant Country'] == ct)&
                                                      (temp_df['Model Year'] > 2015)]),
                            name=ct))
fig.update_layout(
    title_text="Price Distribution v.s. Plant Country",
    xaxis_title="Plant Country",
    yaxis_title="Log-Price",
    legend_title="Plant Country"
)
fig.show()

### Price vs. Brand

In [7]:
make_counts = df01['Make'].value_counts().reset_index()
make_counts.columns = ['Make','Count']

count_t20_makes = list(make_counts.Make[0:25])

df_by_make_count = df01[df01['Make'].isin(count_t20_makes)]



# Assuming you have your data frame 'df_by_make_count' with columns 'Make', 'Price', and 'Year'

year_groups = [list(range(2000, 2010)), list(range(2010, 2013)), list(range(2013, 2016)),
             list(range(2016, 2019)), list(range(2019, 2022)), list(range(2022, 2025))]

brands = list(df_by_make_count['Make'].unique())
df_by_make_count.Make = pd.Categorical(df_by_make_count.Make, 
                      categories=brands,
                      ordered=True)
df_by_make_count = df_by_make_count.sort_values('Make')
brands = list(df_by_make_count['Make'].unique())
colors = ['#8B0000', '#CD0000', '#FF4500', '#FF7F00', '#FFA500', '#FFD700']

buttons = []

fig = go.Figure()
# initial figure
fig.add_trace(go.Bar(x=brands,
                    y=df_by_make_count.groupby('Make')['WxP'].sum().values, visible=True, name="2000-2024", showlegend=True,
                    marker=dict(color='#7B68EE')))

for i, year_group in enumerate(year_groups):
    
    year_data = df_by_make_count[df_by_make_count['Model Year'].isin(year_group)]
    year_data.Make = pd.Categorical(year_data.Make, 
                      categories=brands,
                      ordered=True)
    year_data = year_data.sort_values('Make')
    year_min = min(year_group)
    year_max = max(year_group)
    
    fig.add_trace(go.Bar(x=brands,
                            y=year_data.groupby('Make')['WxP'].sum().values, visible=False, name=f"{year_min}-{year_max}",
                         marker={"color":colors[i]}
                        ))
    visibility = ['legendonly']*len(year_groups)
    visibility[i] = True
    visibility.insert(0, False)
                  
    buttons.append({'label': f"{year_min}-{year_max}", 
                             'method': 'update',
                             'args': [{'visible': visibility, 'title': str(year_group), 'showlegend': True}]
                            })

buttons.insert(0, {'label': '2000-2024',
                            'method': 'update',
                            'args': [{'title': 'All Years', 'showlegend': True,
                                     'visible':[True]+[False]*(len(year_groups))}]})

buttons.insert(len(year_groups)+1, {'label': 'All Year Groups',
                            'method': 'update',
                            'args': [{'title': 'All Year Groups', 'showlegend': True,
                                     'visible':[False]+[True]*(len(year_groups))}]})






fig.update_layout(updatemenus=[dict(active=0,buttons=buttons)],
                  )
# fig.update_layout({'width':1300,
#                    'height':800
#                   }) 
#yaxis_range=[0,300000], 
fig.update_layout(
    title_text="Weighted Average Price v.s. Top 25 Brand and Model Year",
    xaxis_title="Brand",
    yaxis_title="Price (USD)",
    legend_title="Year Group"
)

fig.show()

$\text{Weighted Mean} = \frac{\sum_{i=1}^n w_i X_i}{\sum_{i=1}^n w_i} $

* Price vs. Millage
     * Car Type, Country of Origin, etc.


In [300]:
# scatter plot price vs millage color by country of origin

### Price vs. Mileage

In [8]:
import statsmodels.api as sm
buttons = []

fig = go.Figure()
# initial figure
# fig.add_trace(go.Scatter(x=df_by_make_count['Mileage'],
#                     y=np.log(df_by_make_count['Price']), mode='markers', visible=True, name="2000-2024", showlegend=True,
#                     marker=dict(color='gray')))

# lowess = sm.nonparametric.lowess(np.log(df_by_make_count['Price']), df_by_make_count['Mileage'], frac=0.35)
# fig.add_trace(go.Scatter(x=lowess[:, 0], y=lowess[:, 1], mode='lines', name='LOESS Curve'))

for i, year_group in enumerate(year_groups):
    
    year_data = df_by_make_count[df_by_make_count['Model Year'].isin(year_group)]
    year_data.Make = pd.Categorical(year_data.Make, 
                      categories=brands,
                      ordered=True)
    year_data = year_data.sort_values('Make')
    year_min = min(year_group)
    year_max = max(year_group)
    
    fig.add_trace(go.Scatter(x=year_data['Mileage'],
                            y=np.log(year_data['Price']), mode='markers', visible=True, name=f"{year_min}-{year_max}",
                         marker={"color":colors[i]}, line_shape='spline'
                        ))
    visibility = ['legendonly']*len(year_groups)
    visibility[i] = True
#     visibility.insert(0, False)
                  
    buttons.append({'label': f"{year_min}-{year_max}", 
                             'method': 'update',
                             'args': [{'visible': visibility, 'title': str(year_group), 'showlegend': True}]
                            })

# buttons.insert(0, {'label': '2000-2024',
#                             'method': 'update',
#                             'args': [{'title': 'All Years', 'showlegend': True,
#                                      'visible':[True]+[False]*(len(year_groups))}]})

buttons.insert(0, {'label': 'All Year Groups',
                            'method': 'update',
                            'args': [{'title': 'All Year Groups', 'showlegend': True,
                                     'visible':[False]+[True]*(len(year_groups))}]})





fig.update_layout(updatemenus=[dict(active=0,buttons=buttons)],
                  )
fig.update_traces(marker=dict(opacity=0.6))
# fig.update_layout({'width':1300,
#                    'height':800
#                   }) 
#yaxis_range=[0,300000], 
fig.update_layout(yaxis_range=[7,15], xaxis_range=[0,300*1000])
#yaxis_range=[0,300000], 
fig.update_layout(
    title_text="Log-Price v.s. Mileage and Year",
    xaxis_title="Mileage",
    yaxis_title="Log-Price (USD)",
    legend_title="Year Group"
)

Price decaying exponentially with respect to mileage.

### Price vs. Ratings

In [10]:
from plotly.subplots import make_subplots

score_cols = ['Performance score','Value score','Exterior score',
              'Reliability score','Comfort score','Interior score']

data_2020 = df01[df01['Model Year']>2020].dropna(subset=score_cols)
data_2020[score_cols] = data_2020[score_cols].apply(pd.to_numeric, errors='coerce')

fig = go.Figure()

fig = make_subplots(rows=2, cols=3, shared_yaxes=True)

fig.add_scatter(x=data_2020['Performance score'], y=np.log(data_2020['Price']), mode='markers', name="Performance Score", row=1, col=1)
fig.add_scatter(x=data_2020['Value score'], y=np.log(data_2020['Price']), mode='markers', name="Value Score", row=1, col=2)
fig.add_scatter(x=data_2020['Exterior score'], y=np.log(data_2020['Price']), mode='markers', name="Exterior Score", row=1, col=3)
fig.add_scatter(x=data_2020['Reliability score'], y=np.log(data_2020['Price']), mode='markers', name="Reliability Score", row=2, col=1)
fig.add_scatter(x=data_2020['Comfort score'], y=np.log(data_2020['Price']), mode='markers', name="Comfort Score", row=2, col=2)
fig.add_scatter(x=data_2020['Interior score'], y=np.log(data_2020['Price']), mode='markers', name="Interior Score", row=2, col=3)



fig.update_layout(title="Log-Price v.s. Model Rating Scores")

In [227]:
import statsmodels.api as sm
buttons = []

fig = go.Figure()

colors = ['#8A2BE2', '#9B30FF', '#AB82FF', '#B696FF', '#C1A7FF', '#D0BCFF']

for i, year_group in enumerate(year_groups):
    
    year_data = df_by_make_count[df_by_make_count['Model Year'].isin(year_group)]
    year_data.Make = pd.Categorical(year_data.Make, 
                      categories=brands,
                      ordered=True)
    year_data = year_data.sort_values('Make')
    year_min = min(year_group)
    year_max = max(year_group)
    
    fig.add_trace(go.Scatter(x=year_data['Engine Brake (hp) From'],
                            y=np.log(year_data['Price']), mode='markers', visible=True, name=f"{year_min}-{year_max}",
                         marker={"color":colors[i]}
                        ))
    
    visibility = ['legendonly']*len(year_groups)
    visibility[i] = True
                  
    buttons.append({'label': f"{year_min}-{year_max}", 
                             'method': 'update',
                             'args': [{'visible': visibility, 'title': str(year_group), 'showlegend': True}]
                            })


buttons.insert(0, {'label': 'All Year Groups',
                            'method': 'update',
                            'args': [{'title': 'All Year Groups', 'showlegend': True,
                                     'visible':[False]+[True]*(len(year_groups))}]})



fig.update_layout(updatemenus=[dict(active=0,buttons=buttons)],
                  )
fig.update_traces(marker=dict(opacity=0.6))
# fig.update_layout({'width':1300,
#                    'height':800
#                   }) 
#yaxis_range=[0,300000], 
fig.update_layout(yaxis_range=[7,15], xaxis_range=[0,1000])
#yaxis_range=[0,300000], 
fig.update_layout(
    title_text="Log-Price v.s. Horsepower and Year",
    xaxis_title="Horsepower",
    yaxis_title="Log-Price (USD)",
    legend_title="Year Group"
)

In [233]:
import statsmodels.api as sm
buttons = []

fig = go.Figure()

colors = ['#1F5C55', '#297A71', '#33998D', '#3DB8A9', '#57C7BA', '#75D1C6']
for i, year_group in enumerate(year_groups):
    
    year_data = df_by_make_count[df_by_make_count['Model Year'].isin(year_group)]
    year_data.Make = pd.Categorical(year_data.Make, 
                      categories=brands,
                      ordered=True)
    year_data = year_data.sort_values('Make')
    year_min = min(year_group)
    year_max = max(year_group)
    
    fig.add_trace(go.Scatter(x=year_data['Displacement (L)'],
                            y=np.log(year_data['Price']), mode='markers', visible=True, name=f"{year_min}-{year_max}",
                         marker={"color":colors[i]}
                        ))
    
    visibility = ['legendonly']*len(year_groups)
    visibility[i] = True
                  
    buttons.append({'label': f"{year_min}-{year_max}", 
                             'method': 'update',
                             'args': [{'visible': visibility, 'title': str(year_group), 'showlegend': True}]
                            })


buttons.insert(0, {'label': 'All Year Groups',
                            'method': 'update',
                            'args': [{'title': 'All Year Groups', 'showlegend': True,
                                     'visible':[False]+[True]*(len(year_groups))}]})



fig.update_layout(updatemenus=[dict(active=0,buttons=buttons)],
                  )
fig.update_traces(marker=dict(opacity=0.6))
# fig.update_layout({'width':1300,
#                    'height':800
#                   }) 
#yaxis_range=[0,300000], 
fig.update_layout(yaxis_range=[7,15], xaxis_range=[0,10])
#yaxis_range=[0,300000], 
fig.update_layout(
    title_text="Log-Price v.s. Displacement (L) and Year",
    xaxis_title="Horsepower",
    yaxis_title="Log-Price (USD)",
    legend_title="Year Group"
)

In [238]:
df01[df01['Performance score']==1]

,Make,Model,Model Year,Series,Displacement (L),Engine Brake (hp) From,Engine Number of Cylinders,Plant Country,Body Class,Price,...,Performance score,Value score,Exterior score,Reliability score,Open recall,Country of Origin,MPG City,MPG Highway,Weights,WxP
847,MASERATI,Grecale,2023.0,MODENA,2.0,NaN,4.0,ITALY,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,73101,...,1.0,1.0,1.0,1.0,None,Italy,22.0,29.0,0.093750,6853.218750
4273,MERCEDES-BENZ,Sprinter,2022.0,2500,3.0,NaN,NaN,UNITED STATES (USA),Cargo Van,67995,...,1.0,1.0,1.0,1.0,None,Germany,NaN,NaN,0.005767,392.096181
6534,MITSUBISHI,Eclipse Cross,2023.0,NaN,1.5,152.0,4.0,JAPAN,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,26997,...,1.0,3.0,4.0,2.0,None,Japan,25.0,26.0,0.069767,1883.511628


In [241]:
df01[['Make', 'Model', 'Model Year']][df01['Value score']==1].dropna()

,Make,Model,Model Year
284,LUCID,Air,2023.0
847,MASERATI,Grecale,2023.0
4273,MERCEDES-BENZ,Sprinter,2022.0
9525,LUCID,Air,2023.0


In [242]:
df01[['Make', 'Model', 'Model Year']][df01['Performance score']==1].dropna()

,Make,Model,Model Year
847,MASERATI,Grecale,2023.0
4273,MERCEDES-BENZ,Sprinter,2022.0
6534,MITSUBISHI,Eclipse Cross,2023.0


In [260]:
idx = []
for i, x in enumerate(df01['Reliability score']):
    if x !='—':
        if float(x)==1:
            idx.append(i)

df01[['Make', 'Model', 'Model Year']].iloc[idx]

,Make,Model,Model Year
847,MASERATI,Grecale,2023.0
2081,GMC,Sierra Limited,2022.0
4273,MERCEDES-BENZ,Sprinter,2022.0
4589,GMC,Yukon,2023.0
5277,JEEP,Wrangler,2024.0
6517,GMC,Yukon,2023.0
8043,GMC,Yukon,2023.0


In [261]:
idx = []
for i, x in enumerate(df01['Interior score']):
    if x !='—':
        if float(x)==1:
            idx.append(i)

df01[['Make', 'Model', 'Model Year']].iloc[idx]

,Make,Model,Model Year
847,MASERATI,Grecale,2023.0
2081,GMC,Sierra Limited,2022.0
4273,MERCEDES-BENZ,Sprinter,2022.0


In [262]:
idx = []
for i, x in enumerate(df01['Comfort score']):
    if x !='—':
        if float(x)==1:
            idx.append(i)

df01[['Make', 'Model', 'Model Year']].iloc[idx]

,Make,Model,Model Year
847,MASERATI,Grecale,2023.0
1160,MINI,Countryman,2022.0
4273,MERCEDES-BENZ,Sprinter,2022.0
4829,MINI,Countryman,2022.0
4866,MINI,Countryman,2022.0
7316,MINI,Countryman,2022.0
7382,MINI,Countryman,2022.0
8992,MINI,Countryman,2022.0


In [12]:
df01.columns

Index(['Make', 'Model', 'Model Year', 'Series', 'Displacement (L)',
       'Engine Brake (hp) From', 'Engine Number of Cylinders', 'Plant Country',
       'Body Class', 'Price', 'Exterior color', 'Interior color', 'Drivetrain',
       'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN', 'Stock #',
       'Mileage', 'Title', 'Convenience', 'Entertainment', 'Exterior',
       'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall', 'Country of Origin', 'MPG City',
       'MPG Highway', 'Weights', 'WxP'],
      dtype='object')

In [15]:
def fetch_feature_prop(category, year=0):
    
    feature_col = df01[f'{category}']
    feature_dict = dict()
    
    fnames = []
    for ff in feature_col.dropna():
        for fs in ff.split('&'):
            if not fs in fnames:
                feature_dict[fs] = 0

    if year != 0:
        feature_col = df01[f'{category}'][df01['Model Year'] <= year]
        
    N = len(feature_col)
    for fstring in feature_col.dropna():
        flist = fstring.split('&')
        for f in flist:
            feature_dict[f] += 1/N
    
    out_df = pd.DataFrame([{'Feature': key, 'Proportion': value} for key, value in feature_dict.items()])

    return out_df
    

fetch_feature_prop('Safety')  

,Feature,Proportion
0,Automatic Emergency Braking,0.446026
1,Backup Camera,0.809250
2,Blind Spot Monitor,0.370393
3,Brake Assist,0.933253
4,LED Headlights,0.272746
5,Lane Departure Warning,0.429466
6,Rear Cross Traffic Alert,0.226699
7,Stability Control,0.958901


In [277]:
from ipywidgets import interact

fname='Entertainment'

# def create_interactive_plot(fname):
    
#     testdf = fetch_feature_prop(fname)
#     testdf = testdf.sort_values(by='Proportion', ascending=False)
#     fig = go.FigureWidget()
#     bar = fig.add_bar(x=list(testdf['Feature']))
#     # fig.layout = dict(yaxis=dict(range=[0,4]))

#     @interact(Year=(2000, 2022, 1))
#     def update(Year=2016):

#         updated_df = fetch_feature_prop(fname, Year)
#         updated_df = updated_df.sort_values(by='Proportion', ascending=False)
#         with fig.batch_update():
#             fig.data[0].x=list(updated_df['Feature'])
#             fig.data[0].y=list(updated_df['Proportion'])
#     return fig

# from ipywidgets.embed import embed_minimal_html
# output_filename = 'interactive_plot.html'
# fname = 'Entertainment'

# fig = create_interactive_plot(fname)
# embed_minimal_html(output_filename, views=[fig], title='Interactive Plot')
    


testdf = fetch_feature_prop(fname)
testdf = testdf.sort_values(by='Proportion', ascending=False)
fig = go.FigureWidget()
bar = fig.add_bar(x=list(testdf['Feature']))
# fig.layout = dict(yaxis=dict(range=[0,4]))

@interact(Year=(2000, 2022, 1))
def update(Year=2016):
    
    updated_df = fetch_feature_prop(fname, Year)
    updated_df = updated_df.sort_values(by='Proportion', ascending=False)
    with fig.batch_update():
        fig.data[0].x=list(updated_df['Feature'])
        fig.data[0].y=list(updated_df['Proportion'])
fig

interactive(children=(IntSlider(value=2016, description='Year', max=2022, min=2000), Output()), _dom_classes=(…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '706095cf-e8ed-4952-9e0f-3e96657d8eb2',
     …

In [296]:
fname='Safety'

def plot_bar_interactive(fname):

    # Create figure
    fig = go.Figure()
    year_ranges = np.arange(2000, 2022, 1)

    for year in year_ranges:

        updated_df = fetch_feature_prop(fname, year)
        updated_df = updated_df.sort_values(by='Proportion', ascending=False)

        fig.add_trace(
            go.Bar(x = updated_df['Feature'],
                   y = updated_df['Proportion'],
                visible=False))

    fig.data[10].visible = True

    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)}],
            label=str(year_ranges[i])
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=10,
        currentvalue={"prefix": "Year: "},
        pad={"t": 100},
        steps=steps
    )]

    fig.update_layout(
        sliders=sliders
    )

    fig.show()


In [297]:
plot_bar_interactive('Safety')

In [298]:
plot_bar_interactive('Convenience')

In [299]:
plot_bar_interactive('Entertainment')

In [17]:
category = 'Safety'
feature_col = df01[f'{category}']
N = len(feature_col)
feature_dict = dict()

for fstring in feature_col.dropna():
    flist = fstring.split('&')
    for f in flist:
        if f not in feature_dict.keys():
            feature_dict[f] = 1/N
        else:
            feature_dict[f] += 1/N
pd.DataFrame([{'Feature': key, 'Proportion': value} for key, value in feature_dict.items()])

,Feature,Proportion
0,Automatic Emergency Braking,0.446026
1,Backup Camera,0.809250
2,Blind Spot Monitor,0.370393
3,Brake Assist,0.933253
4,LED Headlights,0.272746
5,Lane Departure Warning,0.429466
6,Rear Cross Traffic Alert,0.226699
7,Stability Control,0.958901


In [204]:
fname='Safety'
testdf = fetch_feature_prop(fname)
testdf = testdf.sort_values(by='Proportion', ascending=False)
fig = go.FigureWidget()
bar = fig.add_bar(x=list(testdf['Feature']))
# fig.layout = dict(yaxis=dict(range=[0,4]))

@interact(Year=(2000, 2022, 1))
def update(Year=2016):
    
    updated_df = fetch_feature_prop(fname, Year)
    updated_df = updated_df.sort_values(by='Proportion', ascending=False)
    with fig.batch_update():
        fig.data[0].x=list(updated_df['Feature'])
        fig.data[0].y=list(updated_df['Proportion'])
fig

interactive(children=(IntSlider(value=2016, description='Year', max=2022, min=2000), Output()), _dom_classes=(…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': 'e7e9315c-45e8-49f0-b2f3-9997a2652356',
     …

In [20]:
category = 'Convenience'
year = 2016
feature_col = df01[f'{category}'][df01['Model Year'] <= year]
N = len(feature_col)
feature_dict = dict()

for fstring in feature_col.dropna():
    flist = fstring.split('&')
    for f in flist:
        if f not in feature_dict.keys():
            feature_dict[f] = 1/N
        else:
            feature_dict[f] += 1/N
pd.DataFrame([{'Feature': key, 'Proportion': value} for key, value in feature_dict.items()])

,Feature,Proportion
0,Adaptive Cruise Control,0.895698
1,Heated Seats,0.345502
2,Heated Steering Wheel,0.086919
3,Keyless Start,0.336375
4,Navigation System,0.397219
5,Power Liftgate,0.169492
6,Remote Start,0.140374
7,Cooled Seats,0.013038
8,Autopilot,0.002173


In [21]:
fname='Convenience'
testdf = fetch_feature_prop(fname)
testdf = testdf.sort_values(by='Proportion', ascending=False)
fig = go.FigureWidget()
bar = fig.add_bar(x=list(testdf['Feature']))
# fig.layout = dict(yaxis=dict(range=[0,4]))

@interact(Year=(2000, 2022, 1))
def update(Year=2016):
    
    updated_df = fetch_feature_prop(fname, Year)
    updated_df = updated_df.sort_values(by='Proportion', ascending=False)
    with fig.batch_update():
        fig.data[0].x=list(updated_df['Feature'])
        fig.data[0].y=list(updated_df['Proportion'])
fig

interactive(children=(IntSlider(value=2016, description='Year', max=2022, min=2000), Output()), _dom_classes=(…

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '28f25280-57cc-41a8-b1ec-2b336ce30b12',
     …

In [38]:
df01.columns

Index(['Make', 'Model', 'Model Year', 'Series', 'Displacement (L)',
       'Engine Brake (hp) From', 'Engine Number of Cylinders', 'Plant Country',
       'Body Class', 'Price', 'Exterior color', 'Interior color', 'Drivetrain',
       'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN', 'Stock #',
       'Mileage', 'Title', 'Convenience', 'Entertainment', 'Exterior',
       'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall', 'Country of Origin', 'MPG City',
       'MPG Highway', 'Weights', 'WxP'],
      dtype='object')

In [145]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# def plot_by_category(fname):
    
#     categorical_col = df01[fname].dropna()
#     cats = categorical_col.unique()
#     total = len(categorical_col)
#     percentages = round(categorical_col.value_counts() / total * 100, 2)

#     # Create the subplots
#     fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'xy'}]],
#                         subplot_titles=('Category Percentage', 'Price Box Plot'))

#     # Create the pie chart
#     fig.add_trace(go.Pie(labels=percentages.index,
#                          values=percentages,
#                          name='Category Percentage'),
#                   row=1, col=1)

#     # Create the box plot
#     for category in cats:
#         fig.add_trace(go.Box(y=df01[df01[fname] == category]['Price'],
#                              name=str(category),
#                              boxpoints='all',
#                              jitter=0.3,
#                              pointpos=-1.8),
#                       row=1, col=2)

#     # Update the layout
#     fig.update_layout(title='Category Percentage and Price Box Plot',
#                       yaxis_title='Price',
#                       boxmode='group')

#     # Show the plot
#     fig.show()

In [195]:
from scipy.special import stdtr
from scipy.stats import ttest_ind, ttest_ind_from_stats


def plot_by_category(fname):
    
    categorical_col = df01[fname].dropna()
    cats = categorical_col.unique()
    total = len(categorical_col)
    percentages = round(categorical_col.value_counts() / total * 100, 2)
    
    a = np.log(df01['Price'][df01[fname]==cats[0]])
    b = np.log(df01['Price'][df01[fname]==cats[1]])

    abar = a.mean()
    avar = a.var(ddof=1)
    na = a.size
    adof = na - 1

    bbar = b.mean()
    bvar = b.var(ddof=1)
    nb = b.size
    bdof = nb - 1

    # Use scipy.stats.ttest_ind_from_stats.
    t, p = ttest_ind_from_stats(abar, np.sqrt(avar), na,
                                  bbar, np.sqrt(bvar), nb,
                                  equal_var=False)
    if p < 0.001:
        p = "<0.001"
    else:
        p = round(p, 3)
    
    color_map = px.colors.qualitative.Set2

    
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'xy'}]],
                       subplot_titles=(f"Percentages of {fname}", f"Distribution Plot (p-value: {p})"))
    
    
    fig.add_trace(go.Pie(labels=cats, textinfo='label+percent',
                     values=percentages, marker=dict(colors=color_map),
                     name='Category Percentage'),
                      
              row=1, col=1)
    
    for i, cat in enumerate(cats):
        fig.add_trace(go.Box(y=np.log(df01[df01[fname] == cat]['Price']),
                             name=str(cat),
                             marker=dict(color=color_map[i])),
                      row=1, col=2)
        

        
    fig.update_layout(title='Category Percentage and Price Box Plot',
                  yaxis_title='Log-Price',
                  boxmode='overlay',
                  showlegend=False)



    fig.show()
    
    return 

In [196]:
plot_by_category('1-owner vehicle')

In [197]:
df01['Accidents or damage'] = df01['Accidents or damage'].replace({'At least 1 accident or damage reported': 'At least 1',
                                                                   'None reported': 'None'})
plot_by_category('Accidents or damage')

In [198]:
df01['Open recall'] = df01['Open recall'].replace({'At least 1 open recall reported':"At least 1"})
df01['Open recall'] = df01['Open recall'].fillna('None')

plot_by_category('Open recall')

In [199]:
plot_by_category('1-owner vehicle')

In [23]:
df01.columns

Index(['Make', 'Model', 'Model Year', 'Series', 'Displacement (L)',
       'Engine Brake (hp) From', 'Engine Number of Cylinders', 'Plant Country',
       'Body Class', 'Price', 'Exterior color', 'Interior color', 'Drivetrain',
       'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN', 'Stock #',
       'Mileage', 'Title', 'Convenience', 'Entertainment', 'Exterior',
       'Safety', 'Seating', 'Accidents or damage', 'Clean title',
       '1-owner vehicle', 'Personal use only', 'Comfort score',
       'Interior score', 'Performance score', 'Value score', 'Exterior score',
       'Reliability score', 'Open recall', 'Country of Origin', 'MPG City',
       'MPG Highway', 'Weights', 'WxP'],
      dtype='object')